In [1]:
%pip install pandas numpy scikit-learn nltk


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')
stop_words = set(stopwords.words('french'))


[nltk_data] Downloading package stopwords to /home/batou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('../data/dataset/donnees_SVM.tsv', sep='\t')
df.head()

,Pertinence,Question,Reponse
0,NON,"Bonjour à toutes et à tous, Je suis en train d...","Bonjour, Je suis entrain de poser env. 100m² d..."
1,NON,Bonjours a Tous Voilà mon projet : ma maison e...,"Si moi, 10 ans chez un fabricant De tous ceux ..."
2,NON,"Bonjour. Mes vacances au ski étant annulées, j...","Oui oui, c'est bien ce que je dis personne ne ..."
3,NON,"Bonjour, Je possède une vieille ferme en pierr...","Bonjour à tous, je suis frappé du même mal que..."
4,NON,"Bonjour, Quelle est la meilleur solution pour ...",pataya a écrit: Merci pour ta réponse. Le ccmi...


In [11]:
data = pd.read_csv('../data/dataset/donnees_SVM.tsv', delimiter='\t')
print(data.head())


  Pertinence                                           Question  \
0        NON  Bonjour à toutes et à tous, Je suis en train d...   
1        NON  Bonjours a Tous Voilà mon projet : ma maison e...   
2        NON  Bonjour. Mes vacances au ski étant annulées, j...   
3        NON  Bonjour, Je possède une vieille ferme en pierr...   
4        NON  Bonjour, Quelle est la meilleur solution pour ...   

                                             Reponse  
0  Bonjour, Je suis entrain de poser env. 100m² d...  
1  Si moi, 10 ans chez un fabricant De tous ceux ...  
2  Oui oui, c'est bien ce que je dis personne ne ...  
3  Bonjour à tous, je suis frappé du même mal que...  
4  pataya a écrit: Merci pour ta réponse. Le ccmi...  


In [8]:
## il va falloir prétraiter tout le texte afin d'avoir des vecteurs cohérents
def preprocess_text(text):
    if isinstance(text, float):  # Gérer les valeurs manquantes
        text = ""
    text = text.lower()  # Convertir en minuscules
    text = ''.join([char for char in text if char not in string.punctuation])  # Supprimer la ponctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Supprimer les stopwords
    return text

In [9]:
## application de notre prétraitement 
data['Question'] = data['Question'].apply(preprocess_text)
data['Reponse'] = data['Reponse'].apply(preprocess_text)

In [14]:
data['Question_Reponse'] = data['Question'] + " " + data['Reponse']

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Question_Reponse'])
y = data['Pertinence']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du classifieur SVM
svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [16]:
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         NON       0.25      0.14      0.18        14
         OUI       0.70      0.82      0.76        34

    accuracy                           0.62        48
   macro avg       0.47      0.48      0.47        48
weighted avg       0.57      0.62      0.59        48

Accuracy: 0.625


Maintenant nous allons appliquer ce svm sur notre dataset afin de voir si selon le SVM la réponse est pertinente ou non

In [18]:
import pandas as pd

# Charger les nouvelles données à partir du fichier TSV
new_data = pd.read_csv('../data/dataset/dataset_complet.tsv', delimiter='\t')

# Prétraiter les nouvelles données
new_data['Question'] = new_data['Question'].apply(preprocess_text)
new_data['Reponse'] = new_data['Reponse'].apply(preprocess_text)

# Extraction des caractéristiques
new_combined_text = new_data['Question'] + ' ' + new_data['Reponse']
new_features = tfidf_vectorizer.transform(new_combined_text)

# Prédiction
new_predictions = svm_model.predict(new_features)

# Affichage des prédictions
for i, prediction in enumerate(new_predictions):
    print(f"Question: {new_data['Question'].iloc[i]}")
    print(f"Réponse: {new_data['Reponse'].iloc[i]}")
    print(f"Pertinence prédite: {prediction}")
    print()

FileNotFoundError: [Errno 2] No such file or directory: 'chemin_vers_votre_fichier.tsv'

In [17]:
new_data = pd.read_csv('../data/dataset/dataset_complet.tsv', delimiter='\t')

# Concaténer les questions et les réponses
new_data['Combined_Text'] = new_data['Question'] + ' ' + new_data['Reponse']

# Prétraiter les nouvelles données
new_data['Combined_Text'] = new_data['Combined_Text'].apply(preprocess_text)

# Extraction des caractéristiques
new_features = tfidf_vectorizer.transform(new_data['Combined_Text'])

# Prédiction
new_predictions = svm_model.predict(new_features)

# Ajouter les prédictions au DataFrame
new_data['Prediction'] = new_predictions

# Afficher le DataFrame avec les prédictions
print(new_data)


NameError: name 'tfidf_vectorizer' is not defined